---
title: "Preparing a retention time data set for machine learning"
date: last-modified
author:
- name: Robbin Bouwmeester
  orcid: 0000-0001-6807-7029
  affiliations:
    - VIB-UGent Center for Medical Biotechnology, VIB, Belgium
    - Department of Biomolecular Medicine, Ghent University, Belgium
---

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ProteomicsML/ProteomicsML/blob/main/tutorials/fragmentation/_nist-1-parsing-spectral-library.ipynb)

# Preparing a retention time data set for machine learning

In [ ]:
from collections import Counter
import os

from pygam import LinearGAM, s, f
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from zipfile import ZipFile

import warnings
warnings.filterwarnings("ignore")

In this tutorial you will learn how to go from MaxQuant evidence files to a data set that is ready for training a retention time prediction model. Retention time is the time it takes for an analyte travels through a column. The travel time depends on the interaction with the stationary phase (usually C18 for proteomics) and mobile phase. Where the mobile phase consists of solvents and changes in physicochemical properties over time with a predefined gradient. The stationary phase remains the same over time. This allows for peptides to elude at different time points, e.g., when it prefers to interact with the mobile phase at a certain percentage of the hydrophobic solvent.

The retention time between different runs can differ significantly and depending on the abundance of the precusor calling the elution apex can be difficult. This means we need to preprocess the data before it is used for machine learning. 

## Reading and formatting input data

We will not need all the columns, define those that might be useful:

In [ ]:
sel_columns = [
    'Raw file', 'Sequence', 'Length', 'Modifications', 'Modified sequence',
    'Retention time','Retention length','Calibrated retention time',
    'Calibrated retention time start', 'Calibrated retention time finish',
    'Retention time calibration', 'Match time difference',"Intensity", "PEP"
]

Read the input files, here a csv. If you read the standard txt you need to modify the read_csv with:

```pd.read_csv("evid_files/PXD028248_evidence_selected_columns.csv",sep="\t",low_memory=False)```

Fill all the NA values with 0.0 and filter on only the most confident identificaitons (PEP <= 0.001).

In [ ]:
with ZipFile('https://github.com/ProteomicsML/ProteomicsML/blob/main/datasets/retentiontime/PXD028248/PXD028248_evidence_selected_columns.zip', 'r') as zipObj:
   zipObj.extractall()
    
evid_df = pd.read_csv("PXD028248_evidence_selected_columns.csv",low_memory=False)
evid_df.fillna(0.0,inplace=True)
evid_df = df[df["PEP"] <= 0.001][sel_columns]

The file in a pandas dataframe looks like this:

In [ ]:
evid_df

As you can see in this example there are many of the same peptidoforms (minus charge) for the different runs. We will want to create a single value for each peptidoform per run in a matrix instead of a single peptidoform+run combo per row.

In [ ]:
retention_dict = {}

# Group by the raw file
for gidx,g in evid_df.groupby("Raw file"):
    # Group by peptidoform and take the mean for each group
    retention_dict[gidx]  = g.groupby("Modified sequence").mean()["Calibrated retention time"].to_dict()
    
#Transform the dictionary in a df where each row is a peptidoform and each column a run
retention_df = pd.DataFrame(retention_dict)

retention_df

We can than have a look at the absence of each peptidoform in all runs (value = absence in that many runs):

In [ ]:
prevelence_peptides = retention_df.isna().sum(axis=1)
print(prevelence_peptides)

We can penalize the absence the absence of highly abundant peptidoforms per run (lower = more abundant peptidoforms present) by taking the dot product of presence/absence in the matrix and the above absence scores:

In [ ]:
score_per_run = retention_df.isna().astype(int).T.dot(prevelence_peptides)
score_per_run.sort_values(ascending=True)

We will use a single run to align all the first experiments against, this is the one with the lowest penalty score:

In [ ]:
run_highest_overlap_score = score_per_run.sort_values(ascending=True).index[0]

## Retention time alignment between runs

The first step after reading and loading the data is to align retention times between runs. Here we will use splines in a GAM for that. The algorithm below follows these steps:

1. Iterate over all runs, sorted by the earlier defined penalty score
2. Obtain the overlapping peptidoforms between runs
3. If there are less than 20 peptidoforms skip that run
4. Divide the overlapping peptides into equidistant bins and enforce a percentage of the bins to be filled with a least one peptidoform (now 200 bins and 75 % occupancy). If requirements are not met skip that run.
5. Fit the GAM with splines between the reference set and the selected run
6. Calculate the error between aligned and values in the reference set. If selected it will run a second stage of the GAM filtering out any data points that were selected to have an error that is too high
7. Assign aligned values to a new matrix
8. Change the reference dataset to be the median of all aligned runs and the initial reference run


In [ ]:
#constraints = "monotonic_inc"
constraints = "none"

# Align parameters
perform_second_stage_robust = True
error_filter_perc = 0.005
num_splines = 150
min_coverage = 0.75
coverage_div = 200
plot_res_every_n = 20
min_overlap = 20

run_highest_overlap_score = score_per_run.sort_values(ascending=True).index[0]

unique_peptides = []
unique_peptides.extend(list(retention_df[retention_df[run_highest_overlap_score].notna()].index))

retention_df_aligned = retention_df.copy()

keep_cols = [run_highest_overlap_score]

error_filter_perc_threshold = max(retention_df[run_highest_overlap_score])*error_filter_perc

# Iterate over runs sorted by a penalty score
for idx,align_name in enumerate(score_per_run.sort_values(ascending=True)[1:].index):
    # Check overlap between peptidoforms
    non_na_sel = (retention_df[align_name].notna()) & (retention_df[run_highest_overlap_score].notna())
    
    # Continue if insufficient overlapping peptides
    if len(retention_df[run_highest_overlap_score][non_na_sel].index) < min_overlap:
        continue
    
    # Check spread of overlapping peptidoforms, continue if not sufficient
    if (len(set(pd.cut(retention_df[align_name][non_na_sel], coverage_div, include_lowest = True))) / coverage_div) < min_coverage:
        continue
    
    # Fit the GAM
    gam_model_cv = LinearGAM(s(0, n_splines=num_splines), constraints=constraints, verbose=True).fit(
                                                            retention_df[align_name][non_na_sel], 
                                                            retention_df[run_highest_overlap_score][non_na_sel])
    
    
    # Plot results alignment
    print(f"---{align_name}---")
    if idx % plot_res_every_n == 0:
        plt.scatter(
                    retention_df[run_highest_overlap_score][non_na_sel],
                    retention_df[align_name][non_na_sel],
                    alpha=0.05,
                    s=10,
                    label="Align set"
                )

        plt.scatter(
                    retention_df[run_highest_overlap_score][non_na_sel],
                    gam_model_cv.predict(retention_df[align_name][non_na_sel]),
                    alpha=0.05,
                    s=10,
                    label="Align set aligend to reference"
                )
        plt.plot(
            [
            min(retention_df[run_highest_overlap_score][non_na_sel]),
            max(retention_df[run_highest_overlap_score][non_na_sel])

            ],
            [
                min(retention_df[run_highest_overlap_score][non_na_sel]),
                max(retention_df[run_highest_overlap_score][non_na_sel])

            ],
            c="black",
            linestyle="--",
            linewidth=1.0
        )
        
        plt.xlabel("Retention time reference set")
        plt.ylabel("Reference - aligned align set")
        leg = plt.legend()
        for lh in leg.legendHandles: 
            lh.set_alpha(1)

        plt.show()
        
        
        plt.scatter(
                    retention_df[run_highest_overlap_score][non_na_sel],
                    retention_df[align_name][non_na_sel]-retention_df[run_highest_overlap_score][non_na_sel],
                    alpha=0.05,
                    s=10
                )

        plt.scatter(
                    retention_df[run_highest_overlap_score][non_na_sel],
                    gam_model_cv.predict(retention_df[align_name][non_na_sel])-retention_df[run_highest_overlap_score][non_na_sel],
                    alpha=0.05,
                    s=10
                )
        
        plt.title("Residual plot")
        
        plt.axhline(
            y = 0.0, 
            color = "black",
            linewidth=1.0,
            linestyle = "--"
        )
        
        plt.xlabel("Retention time reference")
        plt.xlabel("Retention time reference")

        plt.show()
    
    # Calculate errors and create filter that can be used in the second stage
    errors = abs(gam_model_cv.predict(retention_df[align_name][non_na_sel])-retention_df[run_highest_overlap_score][non_na_sel])
    error_filter = errors < error_filter_perc_threshold
    
    # Perform a second stage GAM removing high error from previous fit
    if perform_second_stage_robust:
        gam_model_cv = LinearGAM(s(0, n_splines=num_splines), constraints=constraints, verbose=True).fit(
                                                                retention_df[align_name][non_na_sel][error_filter], 
                                                                retention_df[run_highest_overlap_score][non_na_sel][error_filter])

        if idx % plot_res_every_n == 0:
            plt.scatter(
                        retention_df[run_highest_overlap_score][non_na_sel][error_filter],
                        retention_df[align_name][non_na_sel][error_filter]-retention_df[run_highest_overlap_score][non_na_sel][error_filter],
                        alpha=0.05,
                        s=10
                    )

            plt.scatter(
                        retention_df[run_highest_overlap_score][non_na_sel][error_filter],
                        gam_model_cv.predict(retention_df[align_name][non_na_sel][error_filter])-retention_df[run_highest_overlap_score][non_na_sel][error_filter],
                        alpha=0.05,
                        s=10
                    )
            plt.title("Residual plot second stage GAM")
            plt.axhline(
                y = 0.0, 
                color = "black",
                linewidth=1.0,
                linestyle = "--"
            )
            
            plt.xlabel("Retention time reference set")
            plt.ylabel("Reference - aligned align set")

            plt.show()
    
    # Write alignment to new matrix
    retention_df_aligned.loc[retention_df[align_name].notna(),align_name] = gam_model_cv.predict(retention_df.loc[retention_df[align_name].notna(),align_name])

    unique_peptides.extend(list(retention_df[retention_df[align_name].notna()].index))
    
    keep_cols.append(align_name)
        
    # Create reference set based on aligned retention times
    retention_df["median_aligned"] = retention_df_aligned[keep_cols].median(axis=1)
    run_highest_overlap_score = "median_aligned"

The data points acquired looks as the following vector:

In [ ]:
retention_df_aligned[keep_cols].median(axis=1)

If we look at the standard deviation we can see that this is still relatively large for some peptidoforms:

In [ ]:
plt.hist(retention_df_aligned[keep_cols].std(axis=1),bins=500)
plt.xlabel("Standard deviation retention time")
plt.show()

plt.hist(retention_df_aligned[keep_cols].std(axis=1),bins=500)
plt.xlim(0,7.5)
plt.xlabel("Standard deviation retention time (zoomed)")
plt.show()

In addition to the std there is another factor that can play a big role, the amount of times a peptidoform was observed:

In [ ]:
plt.hist(retention_df_aligned[keep_cols].notna().sum(axis=1),bins=100)
plt.xlabel("Count peptidoforms across runs")
plt.show()

plt.hist(retention_df_aligned[keep_cols].notna().sum(axis=1),bins=100)
plt.xlabel("Count peptidoforms across runs (zoomed)")
plt.xlim(0,20)
plt.show()

If we plot both values against each other we get the following plot (the lines indicate possible thresholds):

In [ ]:
plt.scatter(
    retention_df_aligned[keep_cols].notna().sum(axis=1),
    retention_df_aligned[keep_cols].std(axis=1),
    s=5,
    alpha=0.1
)

plt.ylabel("Standard deviation retention time")
plt.xlabel("Count peptidoforms across runs")

plt.axhline(
    y = 2.0, 
    color = "black",
    linewidth=1.0,
    linestyle = "--"
)

plt.axvline(
    x = 5.0, 
    color = "black",
    linewidth=1.0,
    linestyle = "--"
)

plt.show()

If we set a threshold for a minimum of 5 observations and a maximum standard deviation of 2 we get the following final data set. Here we take the median for each peptidoform across all runs:

In [ ]:
min_observations = 5
max_std = 2.0

observation_filter = retention_df_aligned[keep_cols].notna().sum(axis=1) > min_observations
std_filter = retention_df_aligned[keep_cols].std(axis=1) < max_std

retention_df_aligned[keep_cols][(observation_filter) & (std_filter)].median(axis=1)